## Load the SentenceTransformer and test the embedding is working


In [6]:
from sentence_transformers import SentenceTransformer

embeddingMModel = SentenceTransformer('all-MiniLM-L6-v2')

# def embed_text(text):
#     return embeddingModel.encode(text)
# # Example usage
# if __name__ == "__main__":
#     sample_text = "This is a sample text for embedding."
#     embedding = embed_text(sample_text)
#     print(f"Embedding for the sample text: {embedding[:5]}")

### Load Document and spilt into chunks and then embedding the chunks

In [7]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# load the pdf document
loader = PyPDFLoader("dataset/AI Agents guidebook.pdf")
documents = loader.load()

# split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
chunks = text_splitter.split_documents(documents)
print(f"Number of chunks created: {len(chunks)}")

Ignoring wrong pointing object 899 0 (offset 0)


Number of chunks created: 122


### Embed the chunks 

In [8]:
encoded_chunks = [embeddingMModel.encode(chunk.page_content) for chunk in chunks]
print(f"Embedding for first chunk: {encoded_chunks[0][:5]}")
print(f'Total number of encoded chunks: {len(encoded_chunks)}')
print(f'Length of each encoded chunk: {len(encoded_chunks[1])}')
print(f'Embedding Model Shape: {encoded_chunks[0].shape}')
# Example usage
# if __name__ == "__main__":
#     print(f"Number of chunks created: {len(chunks)}")
#     print(f"Embedding for first chunk: {encoded_chunks[0][:5]}")    

Embedding for first chunk: [-0.05577217 -0.04005972  0.02752373  0.03279826  0.05366068]
Total number of encoded chunks: 122
Length of each encoded chunk: 384
Embedding Model Shape: (384,)


### Save the embedding in Chromdb

In [9]:
import chromadb

# Create a ChromaDB client
client = chromadb.Client()
# Create a collection to store the embeddings
collection = client.get_or_create_collection(name="ai_agents_guidebook")
# Add the chunks , their embeddings and metadata to the collection
for i, chunk in enumerate(chunks):
    collection.add(
        documents=[chunk.page_content],
        embeddings=[encoded_chunks[i]],
        ids=[str(i)],
        metadatas=[{"source": "AI Agents guidebook", "chunk_index": i}]
    )
print(f"Embeddings saved to ChromaDB collection 'ai_agents_guidebook'. {collection.count()} items added.")
print(f"Number of chunks created: {collection}")

Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 4
Insert of existing embedding ID: 4
Add of existing embedding ID: 5
Insert of existing embedding ID: 5
Add of existing embedding ID: 6
Insert of existing embedding ID: 6
Add of existing embedding ID: 7
Insert of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 8
Add of existing embedding ID: 9
Insert of existing embedding ID: 9
Add of existing embedding ID: 10
Insert of existing embedding ID: 10
Add of existing embedding ID: 11
Insert of existing embedding ID: 11
Add of existing embedding ID: 12
Insert of existing embedding ID: 12
Add of existing embedding ID: 13
Insert of existing embedding ID: 13
Add of existing embedding ID: 14
Insert of existing em

Embeddings saved to ChromaDB collection 'ai_agents_guidebook'. 131 items added.
Number of chunks created: Collection(name=ai_agents_guidebook)


### Query and retrieve the vector db here. it is chromadb... Testing done before using LLM


In [10]:
# create a method which will take query and return the relevant chunks
def query_chromadb(query, top_k=3):
    # Embed the query using the same embedding model
    query_embedding = embeddingMModel.encode(query)
    # Query the collection for similar embeddings
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results

       
# Test the method using Query list before using LLM
query_list = [
    "What is MCP Server",
    "What is Kayak Tool?",
    "Can i work from home today?",
    "How to use AutoGPT?",
]



# Example usage with the query list
if __name__ == "__main__":
    print("=" * 60)
    print("🔍 TESTING SEMANTIC SEARCH")
    print("=" * 60)
    for query in query_list:
        print(f"Results for query: {query}")
        results = query_chromadb(query, top_k=3)
        for i, (doc, score, metadata) in enumerate(zip(results['documents'][0], results['distances'][0], results['metadatas'][0])):
            print(f"\n  Result {i+1}:")
            print(f"  📄 Topic: {metadata['source']}")
            print(f"  📏 Distance: {score:.4f} (lower = more similar)")
            print(f"  📖 Text: {doc[:50]}...")

🔍 TESTING SEMANTIC SEARCH
Results for query: What is MCP Server

  Result 1:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.8619 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
Finally, once we have all the a...

  Result 2:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.9035 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#8) Integrate MCP server with...

  Result 3:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.9076 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
#7) Integrate MCP server with C...
Results for query: What is Kayak Tool?

  Result 1:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.5971 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#4) Kayak tool 
A custom Kaya...

  Result 2:
  📄 Topic: AI Agents guidebook
  📏 Distance: 0.6021 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
 
#4) Kayak tool 
A custom Kaya...

  Result 3:
  📄 Topic: AI Agents guidebook
  📏 Distance: 1.0708 (lower = more similar)
  📖 Text: DailyDoseofDS.com 
#5) Browserbase Tool

## Initlaise the SQLite for storing the response generated by LLM

In [50]:
import sqlite3



### Create the Prompt with User Query 

In [ ]:
system_prompt_context = """
You are an AI assistant helping users find information in a document. Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know.
"""
def create_prompt(user_query, context_chunks):
    context_text = "\n\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(context_chunks)])
    prompt = f"{system_prompt_context}\n\n{context_text}\n\nQuestion: {user_query}\nAnswer:"
    return prompt

if __name__ == "__main__":
    user_query = "What is MCP Server?"
    results = query_chromadb(user_query, top_k=3)
    # print(f'Results Retrieved: {results}')
    context_chunks = results['documents'][0]
    # print(f'Context Chunks Retrieved: {len(context_chunks)}')
    # print(f'First Context Chunk: {context_chunks[2]}...\n\n')
    prompt = create_prompt(user_query, context_chunks)
    print("Generated Prompt:")
    print(prompt)   


Generated Prompt:

You are an AI assistant helping users find information in a document. Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know.


Context 1:
DailyDoseofDS.com 
Finally, once we have all the agents and tools deﬁned we set up and kickoﬀ our 
deep researcher crew. 
 
#7) Create MCP Server 
Now, we'll encapsulate our deep research team within an MCP tool. With just a 
few lines of code, our MCP server will be ready. 
Let's see how to connect it with Cursor. 
79

Context 2:
DailyDoseofDS.com 
 
#8) Integrate MCP server with Cursor 
Go to: File → Preferences → Cursor Settings → MCP → Add new global MCP 
server 
In the JSON ﬁle, add what's shown below  
80

Context 3:
DailyDoseofDS.com 
#5) Setup Crew and Kickoﬀ 
We set up and kick oﬀ our ﬁnancial analysis crew to get the result shown below! 
 
#6) Create MCP Server 
Now, we encapsulate our ﬁnancial analyst within an MCP tool and add two more 
tools t

### Call the Groq and OpenAI  LLM Models

In [44]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os
from dotenv import load_dotenv

load_dotenv()
# Initialize LLMs

google_api_key = os.getenv("GOOGLE_API_KEY")
google_llm = None
google_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, google_api_key=google_api_key)

openai_api_key = os.getenv("OPENAI_API_KEY")
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)

groq_api_key = os.getenv("GROQ_API_KEY")
groq_llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0, groq_api_key=groq_api_key)

## Pass the query and context to create a response from LLM

In [47]:
def create_prompt(user_query, context_chunks):
    context_text = "\n\n".join([f"Context {i+1}:\n{chunk}" for i, chunk in enumerate(context_chunks)])
    prompt = f"{system_prompt_context}\n\n{context_text}\n\nQuestion: {user_query}\nAnswer:"
    return prompt

def generate_openai_response(prompt):
    response = openai_llm.invoke(prompt)
    print(f'\n\n Open AI Response: {response.content} *80')
    return response.content

def generate_google_response(prompt):
    response = google_llm.invoke(prompt)
    print(f'\n\n Google LLM Response: {response.content}')
    return response.content

def generate_groq_response(prompt):
    response = groq_llm.invoke(prompt)
    print(f'\n\n Groq LLM Response: {response.content}')
    return response.content



user_query = "Tell me more about multi agent team researcher?"
results = query_chromadb(user_query, top_k=3)

# print(f'Results Retrieved: {results}')
context_chunks = results['documents'][0]
# print(f'Context Chunks Retrieved: {len(context_chunks)}')
# print(f'First Context Chunk: {context_chunks[2]}...\n\n')
prompt = create_prompt(user_query, context_chunks)
# print("Generated Prompt:")
# print(prompt)  

# Generate responses from both LLMs
gemini_llmresponse = generate_google_response(prompt)
groq_response = generate_groq_response(prompt)
               




 Google LLM Response: The context describes a "Multi-agent Deep Researcher" that functions similarly to ChatGPT's deep research feature, providing detailed insights on any topic. It can be built as a local alternative using a specific tech stack. The workflow involves a user submitting a query, a web search agent performing a deep web search, a research analyst verifying and deduplicating the results, and a technical writer crafting a coherent response with citations. This system utilizes multiple agents working together, with one agent for research and another for analysis and writing.


 Groq LLM Response: Based on the provided context, a multi-agent team researcher appears to be a system where multiple agents work together to provide detailed insights on a topic. 

From Context 2, it seems that such a system can be built using a tech stack that includes a platform for deep web research, multi-agent orchestration, and a local server for hosting the research feature. The workflow in

## Save the final Syntentized Response in sqlite

In [53]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import numpy as np

synthesis_prompt = ChatPromptTemplate.from_template("""
Synthesize these two answers into one comprehensive response:

Answer from Gemini LLM:
{response1}

Answer from Groq Llama:
{response2}

Original Question: {question}

Instructions:
1. Combine the best insights from both answers
2. Avoid redundancy
3. Provide a complete, well-organized answer
4. Maintain accuracy and clarity

Provide a single synthesized answer that is better and more complete than both.
""")

chain_synthesis = synthesis_prompt | google_llm | StrOutputParser()
final_response = chain_synthesis.invoke({
    "response1": gemini_llmresponse,
    "response2": groq_response,
    "question": user_query,

})

print(f"✓ Synthesized Final Response:\n{final_response}")

# create a database connection
sqlite_db_conn = sqlite3.connect("documents.db")

# Save it to SQLite
cursor = sqlite_db_conn.cursor()

# 2. Create a table
cursor.execute("""
CREATE TABLE IF NOT EXISTS llm_response (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_query TEXT,
    vector_embedding BLOB,
    llmresponse TEXT
)
""")
# convert the query_vectorbinding into a BLOB using numpy array
queryEmbedding = embeddingMModel.encode(user_query)
print(f'\n\n embedd query is {queryEmbedding[0]}')

embeddingBlob = np.array(queryEmbedding,dtype=np.float32).tobytes()


# 3. Insert a record
cursor.execute("""
INSERT INTO llm_response (user_query, vector_embedding, llmresponse)
VALUES (?, ?, ?)
""", (user_query, embeddingBlob, final_response))

# 4. Commit changes
sqlite_db_conn.commit()

# 5. Verify vector embedding by decode
cursor.execute(
    "SELECT vector_embedding FROM llm_response WHERE user_query = ?",
    (user_query,)
)
blob = cursor.fetchone()[0]

vectorFetchedVal = np.frombuffer(blob, dtype=np.float32)
print(f'Fetched vector binding {vectorFetchedVal[0]}')


print("\n\n Closing Database connection")
sqlite_db_conn.close()



✓ Synthesized Final Response:
A "Multi-agent Team Researcher" is a sophisticated system designed to provide in-depth insights on any given topic, functioning analogously to advanced deep research features found in platforms like ChatGPT. This system can be constructed as a local alternative, leveraging a specific technology stack.

The core of this researcher lies in its multi-agent architecture, where distinct AI agents collaborate to achieve a common goal. The typical workflow begins with a user submitting a query. This query is then handled by a **web search agent** responsible for conducting a comprehensive, deep web search to gather relevant information. Following the search, a **research analyst agent** takes over to meticulously verify the gathered data, deduplicate redundant findings, and ensure the accuracy and reliability of the information. Finally, a **technical writer agent** synthesizes the verified research into a coherent, well-structured response, complete with proper 

## Shorten the summary to 280 characters

In [54]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create a summarization prompt to condense the response to 280 characters
summarization_prompt = ChatPromptTemplate.from_template("""
Summarize the following text in exactly 280 characters or less. 
Make it concise, clear, and informative. Preserve the key message.

Text to summarize:
{text}

Important: Your summary must be exactly 280 characters or less (including spaces and punctuation).
Respond with ONLY the summarized text, no additional explanation.
""")

# Use Google LLM to create the 280-character summary
chain_summarize = summarization_prompt | google_llm | StrOutputParser()

shortened_response = chain_summarize.invoke({
    "text": final_response
})

# Ensure the response doesn't exceed 280 characters (fallback truncation)
if len(shortened_response) > 280:
    shortened_response = shortened_response[:277] + "..."

print(f"\n{'='*80}")
print(f"✓ AI-Generated Summary (280 characters max):")
print(f"{'='*80}")
print(f"{shortened_response}")
print(f"\nOriginal length: {len(final_response)} characters")
print(f"Summary length: {len(shortened_response)} characters")


✓ AI-Generated Summary (280 characters max):
A Multi-agent Team Researcher is a local AI system for deep topic insights. It uses specialized agents: a web search agent gathers info, a research analyst verifies it, and a technical writer crafts a cited response. This collaborative approach ensures accurate, structured res...

Original length: 1764 characters
Summary length: 280 characters


### Send the summarized summary as a email 

In [60]:
from sendgrid import SendGridAPIClient
from sendgrid.helpers.mail import Mail
import os
import pandas as pd

def send_email_with_sendgrid(summarized_text, recipient_email="joyshanker78@gmail.com"):
    """
    Send the summarized response via email using Gmail SMTP
    """
    
    # Get email credentials from environment variables
    sender_email = "joyshanker78@gmail.com" #recipient_email #os.getenv("GMAIL_SENDER_EMAIL")
    #recipient_email = sender_email
    app_password = "Check123$"
    
    if not sender_email or not app_password:
        print("❌ Error: GMAIL_SENDER_EMAIL or GMAIL_APP_PASSWORD not found in .env file")
        print("Please add the following to your .env file:")
        print("  GMAIL_SENDER_EMAIL=your_email@gmail.com")
        print("  GMAIL_APP_PASSWORD=your_app_password")
        return False
    
    try:
        # Create message
        message = MIMEMultipart("alternative")
        message["Subject"] = "AI-Generated Summary: Multi-Agent Team Researcher"
        message["From"] = sender_email
        message["To"] = recipient_email
        
        # Create HTML email body
        html_body = f"""
        <html>
          <body style="font-family: Arial, sans-serif; line-height: 1.6; color: #333;">
            <div style="max-width: 600px; margin: 0 auto; padding: 20px; border: 1px solid #ddd; border-radius: 5px;">
              <h2 style="color: #4CAF50;">✓ AI-Generated Summary</h2>
              
              <p><strong>Topic:</strong> Multi-Agent Team Researcher</p>
              
              <div style="background-color: #f5f5f5; padding: 15px; border-left: 4px solid #4CAF50; margin: 20px 0;">
                <p>{summarized_text}</p>
              </div>
              
              <p><strong>Summary Statistics:</strong></p>
              <ul>
                <li>Character Length: {len(summarized_text)}/280</li>
                <li>Generated using: Google Generative AI (Gemini)</li>
                <li>Timestamp: {pd.Timestamp.now()}</li>
              </ul>
              
              <hr>
              <p style="font-size: 12px; color: #666;">This is an automated email from AI Capstone Assignment. Please do not reply to this email.</p>
            </div>
          </body>
        </html>
        """
        
        # Create plain text version
        text_body = f"""
        AI-Generated Summary
        ====================
        
        Topic: Multi-Agent Team Researcher
        
        {summarized_text}
        
        Character Length: {len(summarized_text)}/280
        Generated using: Google Generative AI (Gemini)
        """
        
        # Attach both versions
        message.attach(MIMEText(text_body, "plain"))
        message.attach(MIMEText(html_body, "html"))
        
        # Send email via Gmail SMTP
        print("\n📧 Sending email...")
        server = smtplib.SMTP_SSL("smtp.gmail.com", 465)
        server.login(sender_email, app_password)
        server.sendmail(sender_email, recipient_email, message.as_string())
        server.quit()
        
        print(f"✓ Email sent successfully to {recipient_email}")
        return True
        
    except smtplib.SMTPAuthenticationError:
        print("❌ Error: Gmail authentication failed. Check your credentials.")
        return False
    except smtplib.SMTPException as e:
        print(f"❌ SMTP error occurred: {e}")
        return False
    except Exception as e:
        print(f"❌ Error sending email: {e}")
        return False

# Send the summarized response
send_email_with_summary(shortened_response,"joyshanker78@gmail.com")

❌ Error: GMAIL_SENDER_EMAIL or GMAIL_APP_PASSWORD not found in .env file
Please add the following to your .env file:
  GMAIL_SENDER_EMAIL=your_email@gmail.com
  GMAIL_APP_PASSWORD=your_app_password


False